In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import kerastuner as kt
from tensorflow.keras.layers import LSTM, Dense, GRU



import warnings
warnings.filterwarnings('ignore')

In [ ]:
folder_path = '/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files'
files_list = glob.glob(folder_path + '/*.xlsx')
dfs = []
for file in files_list:
    df = pd.read_excel(file, parse_dates=(['created_at']))
    dfs.append(df)
    
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.head()

In [ ]:
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'], format='%d/%m/%Y %H:%M')
merged_df["year"] = merged_df["created_at"].dt.year
merged_df["month"] = merged_df["created_at"].dt.month
merged_df["day"] = merged_df["created_at"].dt.day
merged_df["hour"] = merged_df["created_at"].dt.hour
merged_df["minute"] = merged_df["created_at"].dt.minute
merged_df.head()

In [ ]:
null_data = merged_df.isna().sum()
duplicate_rows = merged_df.duplicated()
print('Nulls', null_data)
print('Duplicates', duplicate_rows)

In [ ]:
# Detecting outliers using Z-score or IQR
# Let's focus on the "field1" column for outlier detection
Q1 = merged_df["field1"].quantile(0.25)
Q3 = merged_df["field1"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# Identify and remove outliers
df_no_outliers = merged_df[
    (merged_df["field1"].iloc[:] >= lower_bound) &
    (merged_df["field1"].iloc[:] <= upper_bound)
]
df_no_outliers.head()

In [ ]:
X = df_no_outliers.drop(['field1', 'created_at'], axis=1)
y = df_no_outliers['field1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Applying Standard Scaling to the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Reshape data for LSTM and  gru (samples, timesteps, features)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

In [ ]:
# lstm_model = Sequential([
#     LSTM(128, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True),
#     LSTM(64, activation='relu'),
#     Dense(1)  # Output layer
# ])

# lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# # # Use early stopping to prevent overfitting
# # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# lstm_model.fit(X_train_reshaped, y_train, epochs=100, validation_split=0.2)

# # Evaluate the LSTM model
# y_pred_lstm = lstm_model.predict(X_test_reshaped)

# # Calculate and print R-squared
# r_squared_lstm = r2_score(y_test, y_pred_lstm)
# print("LSTM R-squared:", r_squared_lstm)

In [ ]:
# r_squared = r2_score(y_test, y_pred_lstm)
# mae = mean_absolute_error(y_test, y_pred_lstm)
# rmse = mean_squared_error(y_test, y_pred_lstm, squared=False)
# mse = mean_squared_error(y_test, y_pred_lstm)

# print("TANK2:")
# print("R-squared:", r_squared)
# print("Mean Absolute Error:", mae)
# print("Root Mean Squared Error:", rmse)
# print("Mean Squared Error:", mse)

In [ ]:
# # Define a function to build the LSTM model with hyperparameters
# def build_model(hp):
#     model = keras.Sequential()
#     model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32),
#                    activation=hp.Choice('activation', values=['relu', 'tanh']),
#                    return_sequences=True,
#                    input_shape=(1, X_train_scaled.shape[1])))
#     model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
#                    activation=hp.Choice('activation', values=['relu', 'tanh'])))
#     model.add(Dense(1))

#     # Compile the model
#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
#                   loss='mse',
#                   metrics=['mse'])
#     return model

# # Define the tuner and search space
# tuner = RandomSearch(
#     build_model,
#     objective='val_loss',
#     max_trials=10,  # Number of trials to run
#     directory='my_tuning_directory',  # Directory to save results
#     project_name='my_lstm_tuning'  # Name for this tuning project
# )

# # Perform hyperparameter tuning
# tuner.search(X_train_reshaped, y_train, epochs=100, validation_data=(X_test_reshaped, y_test))

# # Get the best hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# # Build the best model
# best_model = tuner.hypermodel.build(best_hps)

# # Train the best model
# best_model.fit(X_train_reshaped, y_train, epochs=100, validation_data=(X_test_reshaped, y_test))

# # Evaluate the best model
# y_pred = best_model.predict(X_test_reshaped)
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error (MSE) for Best Model:", mse)

In [ ]:
# r_squared = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# mse = mean_squared_error(y_test, y_pred)

# print("TANK2:")
# print("R-squared:", r_squared)
# print("Mean Absolute Error:", mae)
# print("Root Mean Squared Error:", rmse)
# print("Mean Squared Error:", mse)

# GRU

In [ ]:
gru_model = Sequential([
    GRU(128, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True),
    GRU(64, activation='relu'),
    Dense(1)  # Output layer
])

gru_model.compile(optimizer='adam', loss='mean_squared_error')

# Use early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

gru_model.fit(X_train_reshaped, y_train, epochs=100, validation_split=0.2)

# Evaluate the GRU model
y_pred_gru = gru_model.predict(X_test_reshaped)

# Calculate and print R-squared
r_squared_gru = r2_score(y_test, y_pred_gru)
print("GRU R-squared:", r_squared_gru)

In [ ]:
r_squared = r2_score(y_test, y_pred_gru)
mae = mean_absolute_error(y_test, y_pred_gru)
rmse = mean_squared_error(y_test, y_pred_gru, squared=False)
mse = mean_squared_error(y_test, y_pred_gru)

print("TANK2:")
print("R-squared:", r_squared)
print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("Mean Squared Error:", mse)